**Newton Raphson Root Finding Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math
import time

def ImpliedVolatilityNR(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  maxIterations = 1000
  sigNRIV = []
  count = []
  t1 = []
  for t in range(0, len(K)):
    ccNRM = 0
    error = 1
    i = 0
    sigma = math.sqrt(2/tau * abs(np.log(S/K[t])+r*tau))
    st1 = time.time()
    while error > epsilon and i < maxIterations:
      d1 = (np.log(S / float(K[t])) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
      d2 = d1 - sigma * np.sqrt(tau)
      if str(CP).lower()=="c":
        optPrice = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K[t] * np.exp(-r * tau)
      elif str(CP).lower()=="p":
        optPrice = st.norm.cdf(-d2) * K[t] * np.exp(-r * tau) - st.norm.cdf(-d1)*S
      vega = K[t] * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
      g = optPrice - Cm[t]
      g_prim = vega
      sigma = sigma - g / g_prim
      error=abs(g)
      ccNRM = ccNRM + 1
      i = i + 1
    et = time.time()
    tt = et - st1
    t1.append(tt)
    sigNRIV.append(sigma)
    count.append(ccNRM)
  return count, sigNRIV, t1

**Bisection Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math

def ComputeBSM(CP,S,K,sigma,tau,r):
  d1 = (np.log(S / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
  d2 = d1 - sigma * np.sqrt(tau)
  if str(CP).lower()=="c":
    value = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K * np.exp(-r * tau)
  elif str(CP).lower()=="p":
    value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S
  return value

def ImpliedVolatilityBM(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  maxIterations = 1000
  sigBIV = []
  count = []
  t1 = []
  l = []
  u = []
  for t in range(0, len(K)):
    ccBM = 0
    error = 1
    i = 0
    xl = 0.0001
    xu = 2
    st1 = time.time()
    while error > epsilon and i < maxIterations:
      sigma = (xl + xu)/2
      optPrice = ComputeBSM(CP,S,K[t],sigma,tau,r)
      fxl = ComputeBSM(CP,S,K[t],xl,tau,r)
      if (fxl - Cm[t]) * (optPrice - Cm[t]) < 0:
        xu = sigma
      else:
        xl = sigma
      error=abs(optPrice - Cm[t])
      ccBM = ccBM + 1
      i = i + 1
    et = time.time()
    tt = et - st1
    t1.append(tt)
    l.append(xl)
    u.append(xu)
    sigBIV.append(sigma)
    count.append(ccBM)
  return count, sigBIV, t1, l, u

**Trisection Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math

def ComputeBSM(CP,S,K,sigma,tau,r):
  d1 = (np.log(S / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
  d2 = d1 - sigma * np.sqrt(tau)
  if str(CP).lower()=="c":
    value = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K * np.exp(-r * tau)
  elif str(CP).lower()=="p":
    value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S
  return value

def ImpliedVolatilityTrM(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  maxIterations = 1000
  sigTIV = []
  count = []
  t1 = []
  l = []
  u = []
  for t in range(0, len(K)):
    ccTM = 0
    error = 1
    i = 0
    a = 0.0001
    b = 2
    st1 = time.time()
    while True:
      ccTM = ccTM + 1
      i = i + 1
      x1 = (b + 2*a)/3
      x2 = (2*b + a)/3
      fx1 = ComputeBSM(CP,S,K[t],x1,tau,r) - Cm[t]
      fx2 = ComputeBSM(CP,S,K[t],x2,tau,r) - Cm[t]
      if abs(fx1) < abs(fx2):
        x = x1
      else:
        x = x2
      fx = ComputeBSM(CP,S,K[t],x,tau,r) - Cm[t]
      fa = ComputeBSM(CP,S,K[t],a,tau,r) - Cm[t]
      if abs(fx) <= epsilon:
        sigTIV.append(x)
        count.append(ccTM)
        et = time.time()
        tt = et - st1
        t1.append(tt)
        l.append(a)
        u.append(b)
        break
      elif fa * fx1 < 0:
        b = x1
      elif fx1 * fx2 < 0:
        a = x1
        b = x2
      else:
        a = x2
  return count, sigTIV, t1, l, u

**False Position Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math

def ComputeBSM(CP,S,K,sigma,tau,r):
  d1 = (np.log(S / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
  d2 = d1 - sigma * np.sqrt(tau)
  if str(CP).lower()=="c":
    value = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K * np.exp(-r * tau)
  elif str(CP).lower()=="p":
    value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S
  return value

def ImpliedVolatilityRfM(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  maxIterations = 1000
  sigRfIV = []
  count = []
  t1 = []
  l = []
  u = []
  for t in range(0, len(K)):
    ccRfM = 0
    error = 1
    i = 0
    a = 0.0001
    b = 2
    st1 = time.time()
    while True:
      ccRfM = ccRfM + 1
      fa = ComputeBSM(CP,S,K[t],a,tau,r) - Cm[t]
      fb = ComputeBSM(CP,S,K[t],b,tau,r) - Cm[t]
      x = a - (fa*(b-a))/(fb - fa)
      fx = ComputeBSM(CP,S,K[t],x,tau,r) - Cm[t]
      if abs(fx) <= epsilon:
        sigRfIV.append(x)
        count.append(ccRfM)
        et = time.time()
        tt = et - st1
        t1.append(tt)
        l.append(a)
        u.append(b)
        break
      elif fa * fx < 0:
        b = x
      else:
        a = x
  return count, sigRfIV, t1, l, u

**Hybrid Bisection and False Position Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math

def ComputeBSM(CP,S,K,sigma,tau,r):
  d1 = (np.log(S / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
  d2 = d1 - sigma * np.sqrt(tau)
  if str(CP).lower()=="c":
    value = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K * np.exp(-r * tau)
  elif str(CP).lower()=="p":
    value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S
  return value

def ImpliedVolatilityBRfM(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  sigBRfIV = []
  count = []
  t1 = []
  l = []
  u = []
  for t in range(0, len(K)):
    ccBRfM = 0
    error = 1
    i = 0
    a = 0.0001
    b = 2
    a1 = a2 = a
    b1 = b2 = b
    st1 = time.time()
    while True:
      ccBRfM = ccBRfM + 1
      xB = (a+b)/2
      fa = ComputeBSM(CP,S,K[t],a,tau,r) - Cm[t]
      fb = ComputeBSM(CP,S,K[t],b,tau,r) - Cm[t]
      xF =  a - (fa*(b-a))/(fb - fa)
      fxB = ComputeBSM(CP,S,K[t],xB,tau,r) - Cm[t]
      fxF = ComputeBSM(CP,S,K[t],xF,tau,r) - Cm[t]
      if abs(fxB) < abs(fxF):
        x = xB
      else:
        x = xF
      fx = ComputeBSM(CP,S,K[t],x,tau,r) - Cm[t]
      if abs(fx) <= epsilon:
        sigBRfIV.append(x)
        count.append(ccBRfM)
        et=time.time()
        tt = et - st1
        t1.append(tt)
        l.append(a)
        u.append(b)
        break
      if fa * fxB < 0:
        b1 = xB
      else:
        a1 = xB
      if fa * fxF < 0:
        b2 = xF
      else:
        a2 = xF
      a = max(a1,a2)
      b = min(b1,b2)
  return count, sigBRfIV, t1, l, u

**Hybrid Trisection and False Position Method**

In [ ]:
import numpy as np
import scipy.stats as st
import math

def ComputeBSM(CP,S,K,sigma,tau,r):
  d1 = (np.log(S / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
  d2 = d1 - sigma * np.sqrt(tau)
  if str(CP).lower()=="c":
    value = st.norm.cdf(d1) * S - st.norm.cdf(d2) * K * np.exp(-r * tau)
  elif str(CP).lower()=="p":
    value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S
  return value

def ImpliedVolatilityTRfM(CP,Cm,S,K,tau,r):
  epsilon = 0.000001 # initial error
  sigTRfIV = []
  count = []
  t1 = []
  l = []
  u = []
  for t in range(0, len(K)):
    ccTRfM = 0
    error = 1
    i = 0
    a = 0.0001
    b = 2
    a1 = a2 = a
    b1 = b2 = b
    st1 = time.time()
    while True:
      ccTRfM = ccTRfM + 1
      xT1 = (b + 2*a)/3
      xT2 = (2*b + a)/3
      fa = ComputeBSM(CP,S,K[t],a,tau,r) - Cm[t]
      fb = ComputeBSM(CP,S,K[t],b,tau,r) - Cm[t]
      xF =  a - (fa*(b-a))/(fb - fa)
      x = xT1
      fx = fxT1 = ComputeBSM(CP,S,K[t],xT1,tau,r) - Cm[t]
      fxT2 = ComputeBSM(CP,S,K[t],xT2,tau,r) - Cm[t]
      fxF = ComputeBSM(CP,S,K[t],xF,tau,r) - Cm[t]
      if abs(fxT2) < abs(fx):
        x = xT2
      if abs(fxF) < abs(fx):
        x = xF
      if abs(fx) <= epsilon:
        sigTRfIV.append(x)
        count.append(ccTRfM)
        et = time.time()
        tt = et - st1
        t1.append(tt)
        l.append(a)
        u.append(b)
        break
      if fa * fxT1 < 0:
        b1 = xT1
      elif fxT1 * fxT2 < 0:
        a1 = xT1
        b1 = xT2
      else:
        a1 = xT2
      if fa * fxF < 0:
        b2 = xF
      else:
        a2 = xF
      a = max(a1,a2)
      b = min(b1,b2)
  return count, sigTRfIV, t1, l, u